#1D Diffusion Problem

##Mathematical formation:

$$\frac{\partial u}{\partial t} - \triangledown u = -1\text{ in }\Omega$$
where $\Omega = [0,1]$

Initial Condition:
    $$u0 = 1 + x^2$$
Dirichlet Boundary Condition at two ends:
    $$u = 1 + x^2 + t$$
Exact solution: 
    $$u = 1 + x^2 + t$$

##Weak form:

$$\dot u \int_{\Omega}\phi\psi \text{d}x - u\int_{\Omega}\triangledown\cdot(\triangledown \phi)\psi \text{d}x =\int_{\Omega}f\psi \text{d}x   $$

$$\dot u \int_{\Omega}\phi\psi \text{d}x - u\int_{\partial\Omega}n \triangledown \phi \psi \text{d}s + u\int_{\Omega}\triangledown \phi\triangledown\psi \text{d}x =\int_{\Omega}f\psi \text{d}x   $$

$$\dot u \int_{\Omega}\phi\psi \text{d}x + u\int_{\Omega}\triangledown \phi\triangledown\psi \text{d}x =u\int_{\partial\Omega}n \triangledown \phi \psi \text{d}s + \int_{\Omega}f\psi \text{d}x   $$

$$C\dot u + Ku = F$$

In [17]:
'''
1D Diffusion Problem:
    dudt - div(grad(u)) = -1
    
Initial Condition:
    u0 = 1 + x[0]*x[0]

Dirichlet Boundary Condition at two ends:
    u = 1 + x[0]*x[0] + t 
'''

from dolfin import *
import numpy

# Create mesh and define function space
nx = 2
mesh = UnitIntervalMesh(nx)
V = FunctionSpace(mesh, 'Lagrange', 1)

# Define boundary conditions
u0 = Expression('1 + x[0]*x[0] + t', t=0)

class Boundary(SubDomain):  # define the Dirichlet boundary
    def inside(self, x, on_boundary):
        return on_boundary

boundary = Boundary()
bc = DirichletBC(V, u0, boundary)

# Initial condition
u_1 = interpolate(u0, V)
#u_1 = project(u0, V)  # will not result in exact solution!

dt = 0.1      # time step

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
source = Constant(-1)

DEBUG:FFC:Reusing form from cache.


In [18]:
# Capacitance Matrix C
c = u*v*dx
C = assemble(c)
C.array()

DEBUG:FFC:Reusing form from cache.


array([[ 0.16666667,  0.08333333,  0.        ],
       [ 0.08333333,  0.33333333,  0.08333333],
       [ 0.        ,  0.08333333,  0.16666667]])

In [19]:
# Stiffness Matrix K
k = inner(nabla_grad(u), nabla_grad(v))*dx
K = assemble(k)
K.array()

DEBUG:FFC:Reusing form from cache.


array([[ 2., -2.,  0.],
       [-2.,  4., -2.],
       [ 0., -2.,  2.]])

In [20]:
# Force Vector F
f = source*v*dx
F = assemble(f)
F.array()

DEBUG:FFC:Reusing form from cache.


array([-0.25, -0.5 , -0.25])

In [21]:
# Boundary condition vector G
zero_vector = Constant(0.0)
g = zero_vector*v*ds
G = assemble(g)
G.array()

DEBUG:FFC:Reusing form from cache.


array([ 0.,  0.,  0.])